In [2]:
import numpy as np
import pandas as pd
import random
from collections import Counter, defaultdict
from sklearn.model_selection import KFold, StratifiedKFold

In [3]:
train = pd.read_csv("/code/data/processed/proceeded_train_<built-in method now of type object at 0x7efbfea9bda0>.csv", index_col=0)

In [4]:
train.head()

,Clip,Activity,Assessment,Game,session_title,count_correct_attempts,count_uncorrect_attempts,count_accuracy,duration_mean,accuracy_group,mean_accuracy_group,0,1,2,3,installation_id,count_actions
0,11,3,0,4,19,0,0,0.0,0.00,3,0.0,0,0,0,0,0006a69f,647
1,14,4,1,6,38,1,0,1.0,39.00,0,3.0,0,0,0,1,0006a69f,1143
2,14,4,2,6,19,1,11,0.5,65.50,3,1.5,1,0,0,1,0006a69f,1230
3,24,9,4,10,19,2,11,0.5,41.25,2,1.5,2,0,0,2,0006a69f,2159
4,28,10,5,13,38,3,12,0.5,39.20,3,1.6,2,0,1,2,0006a69f,2586


In [5]:
def stratified_group_k_fold(X, y, groups, k, seed=None):
    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices

def get_distribution(y_vals):
    y_distr = Counter(y_vals)
    y_vals_sum = sum(y_distr.values())
    return [f'{y_distr[i] / y_vals_sum:.2%}' for i in range(np.max(y_vals) + 1)]

In [7]:
train_x = train.drop('accuracy_group', axis=1)
train_y = train['accuracy_group']
groups = np.array(train['installation_id'])

In [10]:
distrs = [get_distribution(train_y)]
index = ['training set']

for fold_ind, (dev_ind, val_ind) in enumerate(stratified_group_k_fold(train_x, train_y, groups, k=5, seed=3)):
    dev_y, val_y = train_y[dev_ind], train_y[val_ind]
    dev_groups, val_groups = groups[dev_ind], groups[val_ind]
    assert len(set(dev_groups) & set(val_groups)) == 0
    
    distrs.append(get_distribution(dev_y))
    index.append(f'development set - fold {fold_ind}')
    distrs.append(get_distribution(val_y))
    index.append(f'validation set - fold {fold_ind}')

In [11]:
display('Distribution per class:')
pd.DataFrame(distrs, index=index, columns=[f'Label {l}' for l in range(np.max(train_y) + 1)])

'Distribution per class:'

,Label 0,Label 1,Label 2,Label 3
training set,23.91%,13.63%,12.46%,50.00%
development set - fold 0,23.91%,13.62%,12.47%,50.00%
validation set - fold 0,23.91%,13.65%,12.46%,49.99%
development set - fold 1,23.90%,13.63%,12.46%,50.00%
validation set - fold 1,23.91%,13.62%,12.46%,50.00%
development set - fold 2,23.90%,13.63%,12.46%,50.00%
validation set - fold 2,23.91%,13.62%,12.46%,50.00%
development set - fold 3,23.91%,13.63%,12.47%,49.99%
validation set - fold 3,23.90%,13.63%,12.44%,50.03%
development set - fold 4,23.91%,13.63%,12.46%,50.00%
